In [1]:
import os, shutil
import rasterio as rio
import numpy
from matplotlib import pyplot
from rasterio.plot import show
import numpy as np

In [2]:
loc_orthos = "../EPCExtent_30cm/Orthos/"
loc_msavi = "../EPCExtent_30cm/Orthos/MSAVI"
os.makedirs(loc_msavi, exist_ok=True)
orthos_files = [os.path.join(loc_orthos,f) for f in os.listdir(loc_orthos) if f.endswith(".tif")]

In [8]:
count=0
for file in orthos_files:
    count+=1
    ofile = os.path.join(loc_msavi, os.path.basename(file).replace(".tif","_MSAVI.tif"))
    if os.path.exists(ofile):
        continue
    with rio.open(file) as ras:
        print(f"{count} - Starting file {ofile}")
        red = ras.read(1).astype(rio.float32)
        green = ras.read(2).astype(rio.float32)
        blue =  ras.read(3).astype(rio.float32)
        nir = ras.read(4).astype(rio.float32)
        sumzero = red+green+blue+nir
    
        msavi = ((2 * nir + 1) - np.sqrt(np.square(2 * nir + 1) - (8 * (nir - red)))) / 2
        kwargs = ras.profile
        kwargs.update(
            count=1,
            dtype=rio.float32,
            nodata=-9999
        )
        
        msavi = np.where(sumzero == 0, -9999, msavi)
        with rio.open(ofile, 'w', **kwargs) as oras:
            oras.write(msavi,1)
    
print(f"Finished writing all NDVI files to {loc_msavi}")

Finished writing all NDVI files to ../EPCExtent_30cm/Orthos/MSAVI
